In [1]:
import camelot
import pandas as pd
import openpyxl
import os
# tables = camelot.read_pdf('epd1388_2021.pdf', pages='10',process_background=True)
# tables = camelot.read_pdf('./IBU-09-20/New/14632.pdf', pages='6', split_text=True,table_areas=["69.60548391889084,500.088593765987,525.3099336281931,403.96614296894364"])
# print(len(tables))

# tables = camelot.read_pdf('./New/14699.pdf', pages='all')
# print(len(tables))

In [2]:
columns = {                                                                                                   
  "impactGWP100_kgCO2e": "(^Global warming potential \xad total$)|(^GWP-total$)|(^GWP$)|(^Global warming potential$)|(^Globales Erwärmungspotenzial ­ total$)|(^Globales Erwärmungspotenzial$)",
  "impactODP_kgCFC11e": "(^ODP$)|(^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)",
  "impactAP_kgSO2e": "(^AP$)|(^AP Air$)|(^Versauerungspotenzial von Boden und Wasser$)|(^Acidification potential of land and water$)",
  "impactEP_kgPO4e": "(^Eutrophication, fraction of nutrients reaching freshwater \nend compartment$)|(^EP-freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)",
  "impactPOCP_kgEthenee": "(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical \noxidants)",
  "impactADPElements_kgSbe": "(^Abiotic depletion potential for non\xadfossil resources$)|(^ADPE$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nnicht fossile Ressourcen$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nnicht fossile Ressourcen$)|(^Abiotic depletion potential for non-fossil resources)",
  "impactADPFossilFuels_MJ": "(^ADPF$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nfossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nfossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)",
    
  "renewablesUsedAsEnergy_MJ": "(^PERE$)|(^Erneuerbare Primärenergie als Energieträger$)|(^Renewable primary energy as energy carrier$)",
  "renewablesUsedAsMaterial_MJ": "(^PERM$)|(^Erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Renewable primary energy resources as material utilization$)",
  "embodiedRenewableEnergy_MJ": "(^PERT$)|(^Total erneuerbare Primärenergie$)|(^Total use of renewable primary energy resources$)",
  "nonRenewablesUsedAsEnergy_MJ": "(^Non\xadrenewable primary energy as energy carrier$)|(^PENRE$)|(^Nicht-erneuerbare Primärenergie als Energieträger$)|(^Nicht\xaderneuerbare Primärenergie als Energieträger$)|(^Non-renewable primary energy as energy carrier$)",
  "nonRenewablesUsedAsMaterial_MJ": "(^Non\xadrenewable primary energy as material utilization$)|(^PENRM$)|(^Nicht-erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Nicht\xaderneuerbare Primärenergie zur stofflichen Nutzung$)|(^Non-renewable primary energy as material utilization$)",
  "embodiedEnergy_MJ": "(^PENRT$)|(^Total nicht erneuerbare Primärenergie$)|(^Total use of non-renewable primary energy resources$)|(^Total use of non\xadrenewable primary energy resources$)",
  "recyclingMaterialUse_kg": "(^SM$)|(^Einsatz von Sekundärstoffen$)|(^Use of secondary material$)",
  "renewableRecylingFuelUse_MJ": "(^RSF$)|(^Erneuerbare Sekundärbrennstoffe$)|(^Use of renewable secondary fuels$)",
  "nonRenewableRecylingFuelUse_MJ": "(^Use of non\xadrenewable secondary fuels$)|(^NRSF$)|(^Nicht-erneuerbare Sekundärbrennstoffe$)|(^Nicht\xaderneuerbare Sekundärbrennstoffe$)|(^Use of non-renewable secondary fuels$)",
  "cleanWaterNetUse_m3": "(^FW$)|(^Nettoeinsatz von Süßwasserressourcen$)|(^Use of net fresh water$)",
    
  "wasteHazardous_kg": "(^HWD$)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)",
  "wasteNonHazardous_kg": "(^Non\xadhazardous waste disposed$)|(^NHWD$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)",
  "wasteRadioactive_kg": "(^RWD$)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)",
  "reusableMaterialsOutput_kg": "(^CRU$)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re\xaduse$)",
  "recyclableMaterialsOutput_kg": "(^MFR$)|(^Stoffe zum Recycling$)|(^Materials for recycling$)",
  "energyMaterialsOutput_kg": "(^MER$)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)",
  "exportedEnergyOutput_MJ_electric": "(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)",
  "exportedEnergyOutput_MJ_thermal": "(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)",
  "exportedEnergyOutput_MJ": "(^EE$)",
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}

headers = [
    "impactGWP100_kgCO2e",
    "impactODP_kgCFC11e",
    "impactAP_kgSO2e",
    "impactEP_kgPO4e",
    "impactPOCP_kgEthenee",
    "impactADPElements_kgSbe",
    "impactADPFossilFuels_MJ",
    
    "renewablesUsedAsEnergy_MJ",
    "renewablesUsedAsMaterial_MJ",
    "embodiedRenewableEnergy_MJ",
    "nonRenewablesUsedAsEnergy_MJ",
    "nonRenewablesUsedAsMaterial_MJ",
    "embodiedEnergy_MJ",
    
    "recyclingMaterialUse_kg",
    "renewableRecylingFuelUse_MJ",
    "nonRenewableRecylingFuelUse_MJ",
    "cleanWaterNetUse_m3",
    "wasteHazardous_kg",
    "wasteNonHazardous_kg",
    "wasteRadioactive_kg",
    "reusableMaterialsOutput_kg",
    "recyclableMaterialsOutput_kg",
    "energyMaterialsOutput_kg",
    "exportedEnergyOutput_MJ_thermal",
    "exportedEnergyOutput_MJ_electric",
    "exportedEnergyOutput_MJ",
    
    "traciGWP_kgCO2e",
    "traciODP_kgCFC11e",
    "traciAP_kgSO2e",
    "traciEP_kgNe",
    "traciPOCP_kgO3e",
    "traciNRPE_MJ"
]

In [7]:
def epd_number(file_name):
#     tables = camelot.read_pdf('./New/' + file_name, pages='1')
#     for table in tables:
#         display(table.df)
    return 'XXXXXXXXXXXXxxxxxx'

In [8]:

def make_excel_file(files_data):
    book = openpyxl.Workbook()
    filename = 'test_ibu.xlsx'
    book.save(filename)

    sheet = book['Sheet']
    sheet.cell(row=1, column=1).value = 'Filename'
    sheet.cell(row=1, column=2).value = 'EPD_Number'
    sheet.cell(row=1, column=3).value = 'DownloadLink'
    sheet.cell(row=1, column=4).value = 'Stages'
    
    count = 5
    for header in headers:
        sheet.cell(row=1, column=count).value = header
        count += 1
    
    current_row = 2
    for file_data in files_data:
        print(file_data["file_name"])
        print(file_data["stages"])
        print("*******************")
        df = file_data["dataFrame"]
        base_added = False
        
        for stage in file_data["stages"]:
            if not base_added:
                sheet.cell(row=current_row, column=1).value = file_data["file_name"]
                sheet.cell(row=current_row, column=2).value = epd_number(file_data["file_name"])
                sheet.cell(row=current_row, column=3).value = '/mnt/data/Downloads/2021-10-18/IBU/New/' + file_data["file_name"]
                base_added = True

            sheet.cell(row=current_row, column=4).value = stage
            header_counter = 0
            for header in headers:
                
#                 i = file_data["stages"].index(stage) + 2
#                 sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
#                 value = float(df[df[0].str.contains(columns2[header])][i].to_list()[0])
#                 #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
# #                 sheet.cell(row=current_row, column= 5 + header_counter).value = value
                try:
                    num_of_columns = len(df.columns)                    
                    i = file_data["stages"].index(stage) + (num_of_columns - len(file_data["stages"]))
                    sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
                    replaced = df[df[0].str.contains(columns[header])][i].to_list()[0].replace(',', '.')
                    if not replaced == 'IND':
                        try:
                            value = float(replaced)
                        except:
                            try:
                                value = float(replaced.replace('\xad', '-'))
                            except:
                                value = float(replaced.replace('\n', ''))
                    else:
                        value = replaced
                        
                    #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
                    sheet.cell(row=current_row, column= 5 + header_counter).value = value
                except:
                    print("**************************************ERRROR*****************************")
                    print(file_data["file_name"])
                    print(stage)
                    print(columns[header])
                header_counter= header_counter +1
            current_row = current_row + 1
        current_row = current_row + 1
            
    book.save(filename)
    book.close()

In [9]:
def clean_df(df):
    transformed = df.applymap(lambda x: x.split(' ') if ('E-' in x) or ('E+' in x) or (x == '') else x)
    for i in range(transformed.shape[0]):
        row = transformed.iloc[[i]]
        for index, element in row.iloc[0].items():
            if element == ['']:
                for index_s, search in row.iloc[0].items():
                    if (len(search) > 1) and ('[kg' not in search) and (type(search) == list):
                        if index < index_s:        
                            transformed.loc[i,index] = [search[0]]
                            row.iloc[0][index] = [search[0]]
                            del search[0]
                        else: 
                            transformed.loc[i,index] = [search[1]]                            
                            row.iloc[0][index] = [search[1]]
                            del search[1]
                        break
    return transformed.applymap(lambda x: x[0] if type(x) == list else x)



In [10]:
files = os.listdir('./1')
# files = ['15230.pdf', '14983.pdf.pdf', '14956.pdf']
r = []
for file in files:
    print("************ FILE NAME: " + file + " ****************")
    tables = camelot.read_pdf('./1/' + file, pages='all')
    dataFrames = []

    file_info = {}
    stages = []
    for table in tables:
        df = table.df
        stages_added = True
        stages = ['A1', 'A2', 'A3', 'C1', 'C2', 'C3', 'C4', 'Total', 'D']
        gwp = ['GWP-total', 'Global warming potential \xad total', 'Global warming potential', 'GWP', 'Globales Erwärmungspotenzial ­ total', 'Globales Erwärmungspotenzial']
        for g in gwp:
            if (g in df.values) and ('TRACI 2.1 Impact Assessment' not in df.values):
                if not stages_added:
                    try:

                        a = list(df[df.eq('Unit').any(1)].values[0])
                        unit_index = a.index('Unit') + 1
                        stages = a[unit_index:]
                    except:
                        a = list(df[df.eq('Einheit').any(1)].values[0])
                        unit_index = a.index('Einheit') + 1
                        stages = a[unit_index:]
                    stages_added = True
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                print(1)
                dataFrames.append(df)


        pere = ['PERE', 'Renewable primary energy as energy carrier', 'Erneuerbare Primärenergie als Energieträger']
        for p in pere:
            if p in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                dataFrames.append(df)
                print(2)

        hwd = ['HWD', 'Hazardous waste disposed', 'Gefährlicher Abfall zur Deponie']
        for h in hwd:
            if h in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                dataFrames.append(df)
                print(3)

                
    result = pd.concat(dataFrames)

    file_info["dataFrame"] = result

        
    file_info["file_name"] = file
    file_info["stages"] = stages
    
    r.append(file_info)
    

make_excel_file(r)


#r

************ FILE NAME: epd5036.pdf ****************
1
2
************ FILE NAME: epd5039.pdf ****************
1
2
************ FILE NAME: epd5040.pdf ****************
1
2
epd5036.pdf
['A1', 'A2', 'A3', 'C1', 'C2', 'C3', 'C4', 'Total', 'D']
*******************
**************************************ERRROR*****************************
epd5036.pdf
A1
(^Abiotic depletion potential for non­fossil resources$)|(^ADPE$)|(^Potenzial für die Verknappung abiotischer Ressourcen ­ 
nicht fossile Ressourcen$)|(^Potenzial für die Verknappung abiotischer Ressourcen - 
nicht fossile Ressourcen$)|(^Abiotic depletion potential for non-fossil resources)
**************************************ERRROR*****************************
epd5036.pdf
A1
(^ADPF$)|(^Potenzial für die Verknappung abiotischer Ressourcen ­ 
fossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - 
fossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)
**************************************ERRROR*****

C:\Users\mirol\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


**************************************ERRROR*****************************
epd5036.pdf
A3
(^HWD$)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)
**************************************ERRROR*****************************
epd5036.pdf
A3
(^Non­hazardous waste disposed$)|(^NHWD$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)
**************************************ERRROR*****************************
epd5036.pdf
A3
(^RWD$)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)
**************************************ERRROR*****************************
epd5036.pdf
A3
(^CRU$)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re­use$)
**************************************ERRROR*****************************
epd5036.pdf
A3
(^MFR$)|(^Stoffe zum Recycling$)|(^Materials for recycling$)
**************************************ERRROR*****************************
epd5036.pdf
A3
(^MER$)|(^Stoffe für die Energierückgewin

A3
(^ADPF$)|(^Potenzial für die Verknappung abiotischer Ressourcen ­ 
fossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - 
fossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)
**************************************ERRROR*****************************
epd5039.pdf
A3
(^HWD$)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)
**************************************ERRROR*****************************
epd5039.pdf
A3
(^Non­hazardous waste disposed$)|(^NHWD$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)
**************************************ERRROR*****************************
epd5039.pdf
A3
(^RWD$)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)
**************************************ERRROR*****************************
epd5039.pdf
A3
(^CRU$)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re­use$)
**************************************ERRROR************

**************************************ERRROR*****************************
epd5040.pdf
C1
(^MFR$)|(^Stoffe zum Recycling$)|(^Materials for recycling$)
**************************************ERRROR*****************************
epd5040.pdf
C1
(^MER$)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)
**************************************ERRROR*****************************
epd5040.pdf
C1
(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)
**************************************ERRROR*****************************
epd5040.pdf
C1
(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd5040.pdf
C1
(^EE$)
**************************************ERRROR*****************************
epd5040.pdf
C1
8
**************************************ERRROR*****************************
epd5040.pdf
C1
12
**************************************ERRROR*****************************
epd

In [9]:
files

['epd5036.pdf', 'epd5039.pdf', 'epd5040.pdf']

In [34]:
d = r[5]['dataFrame']
d

,0,1,2,3,4,5,6,7
0,Core Indicator,Unit,A1-A3,C1,C2,C3,C4,D
1,Global warming potential ­ total,[kg CO2­Eq.],2.54E+1,0.00E+0,3.96E­2,0.00E+0,4.22E­1,0.00E+0
2,Global warming potential ­ fossil fuels,[kg CO2­Eq.],2.58E+1,0.00E+0,3.93E­2,0.00E+0,1.97E­1,0.00E+0
3,Global warming potential ­ biogenic,[kg CO2­Eq.],­4.21E­1,0.00E+0,­4.67E­5,0.00E+0,2.24E­1,0.00E+0
4,GWP from land use and land use change,[kg CO2­Eq.],1.76E­2,0.00E+0,3.21E­4,0.00E+0,5.78E­4,0.00E+0
5,Depletion potential of the stratospheric ozone...,[kg CFC11­Eq.],8.75E­13,0.00E+0,7.75E­18,0.00E+0,7.63E­16,0.00E+0
6,"Acidification potential, accumulated exceedance",[mol H+­Eq.],7.79E­2,0.00E+0,7.74E­5,0.00E+0,1.40E­3,0.00E+0
7,"Eutrophication, fraction of nutrients reaching...",[kg PO4­Eq.],9.16E­5,0.00E+0,1.17E­7,0.00E+0,3.30E­7,0.00E+0
8,"Eutrophication, fraction of nutrients reaching...",[kg N­Eq.],2.13E­2,0.00E+0,3.21E­5,0.00E+0,3.63E­4,0.00E+0
9,"Eutrophication, accumulated exceedance",[mol N­Eq.],1.54E­1,0.00E+0,3.64E­4,0.00E+0,3.99E­3,0.00E+0


In [51]:
d[0][6].tolist()

['Acidification potential, accumulated exceedance',
 'Total use of non\xadrenewable primary energy resources',
 'Materials for energy recovery']